In [39]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.models import Sequential

In [40]:
# importing keras pre-trained models
from tensorflow.keras.applications import Xception,InceptionV3,InceptionResNetV2,ResNet50

In [41]:


train_dir='../input/inaturalist/inaturalist_12K/train'
test_dir='../input/inaturalist/inaturalist_12K/val'


In [42]:
!pip install wandb
import wandb
wandb.login()

In [43]:
# configure sweep parameters
sweep_config = {
    
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  "parameters": {
                   'base_model': {'values': ['inceptionv3','inceptionresnetv2','resnet50','Xception']},
                   'data_augment': {'values': [False, True]},
                   'batch_norm': {'values': [False, True]}, 
                   "batch_size": { "values": [32, 64] },
                   'dropout_rate': {'values': [0.2, 0.3]},
                   'dense_size': {'values': [128,256]},
                   'epochs': {'values': [5,10]}, 
                }
    
}

sweep_id = wandb.sweep(sweep_config,project='CS6910 Assignment2 partB', entity='go4rav')

In [44]:
def train():
    
    #initialising wandb
    wandb.init()
     
    # loading configured parameters
    CONFIG=wandb.config
    dropout_rate = CONFIG.dropout_rate
    dense_size = CONFIG.dense_size
    data_augment = CONFIG.data_augment
    batch_norm = CONFIG.batch_norm
    batch_size = CONFIG.batch_size
    epochs = CONFIG.epochs
    base_model = CONFIG.base_model
    input_shape = (256,256,3)
    
    # fixing image width and height
    img_height = 256
    img_width =  256

    wandb.run.name =  "_bm_"+base_model+"_ds_"+str(dense_size)+"_dr_"+str(dropout_rate)+"_bs_" + str(batch_size)+ "_da_" + str(data_augment) + "_bn_"+str(batch_norm)+"_ne_"+str(epochs)
 
    # inceptionv3 pretrained model
    if base_model == 'inceptionv3':
        base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
        
    # inceptionresnetv2 pretrained model
    if base_model == 'inceptionresnetv2':
        base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    
    # resnet50 pretrained model
    if base_model == 'resnet50':
        base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
      
    # Xception pretrained model
    if base_model == 'Xception':
        base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  
     
    # Freezing base model layers
    for layers in base_model.layers:
        layers.trainable = False
        
    with tf.device('/device:GPU:0'):
        tf.keras.backend.clear_session()
        model = tf.keras.Sequential([
          tf.keras.Input(shape=(img_height, img_width,3,)),
          base_model,
          Flatten(),
          Dense(dense_size,activation='relu'),
        ])

        # batch normalization
        if batch_norm == True:
            model.add(BatchNormalization())
        
        # Adding dropout
        model.add(Dropout(dropout_rate))
        model.add(Dense(dense_size, activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(10 ,activation='softmax'))
            
        # Data augmentation
        if data_augment == True:
            datagen= ImageDataGenerator(
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            rescale=1.0 / 255,
            validation_split=0.1,
            )
        else:
            datagen = ImageDataGenerator(
            rescale=1.0 / 255,
            validation_split=0.1,
            )
            
        train_data = datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size= batch_size,
        class_mode='categorical',
        shuffle=True,
        subset='training',
        seed=100,
        )
        
        
        valid_data = datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        class_mode='categorical',
        shuffle=True,
        subset='validation',
        seed=100,
        )
        
        optimiser = tf.keras.optimizers.Adam(learning_rate=0.0004)
        # Model compilation
        model.compile(optimizer=optimiser, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
        # Model Fitting
        model.fit(train_data, epochs=epochs, validation_data=valid_data, callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5),
                                            wandb.keras.WandbCallback()])
        
        # Saving our model
        model.save('./part2/TrainedModel/'+wandb.run.name)
        wandb.finish()
 


In [45]:
wandb.agent(sweep_id, train ,count = 20)
